# SynthPop example use/tutorial notebook
Written by Macy Huston

Start by importing the package

In [ ]:
import synthpop 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Example 1

#### Step 1: SynthPop model object setup

In this step, we set up the configuration of our Galactic model.

There are a number of ways you can initialize a SynthPop model. Let's take a look at the easiest one first, where you just use a specific config file. This is a great option for testing to make sure you can run SynthPop properly.

In this case, any parameters not controlled by your config will be pulled from the _default.synthpop_conf.

This step is very fast.

The output will show the configuration files used, and a dictionary that could be copied into a new config file to fully recreate your model version.

In [ ]:
mod1 = synthpop.SynthPop('my_config.synthpop_conf')

#### Step 2: Initialize populations

In this step, SynthPop will process your configuration and initialize all of the modules for each of your Populations. 

This will take several seconds, possibly more if you have modules that e.g. load large extinction map files.

The output will log each population as it gets read in from its file and initialized

In [ ]:
mod1.init_populations()

#### Step 3: process your configured locations

In this step, SynthPop will generate a catalog of stars for each sightline specified in your configration. 

This step's runtime depends on your solid angle, the density of the region of sky selected, and a number of other factors. It may take several seconds to several minutes, typically less than a minute for the use cases tested.

The output will log the generation for each sightline and population therein.

In [ ]:
mod1.process_all()

#### Step 4: examine your catalog

The output is a csv catalog of stars with their positions, velocities, properties, and photometry.

In [ ]:
cat1 = pd.read_csv('synthpop/outputfiles/default_synthpop/besancon_Robin2003_l1.000_b1.000.csv')
cat1

## Example 2

#### Step 1: SynthPop model object setup

Here's another option for setting up your model, where you use your own default configuration and put in a few parameters directly. Here, we override the defaults for the extinction map and synthetic photometry, and give our output files a distinct name.

I like to use this method when I'm testing how changing a single factor (like the extinction map) changes the output for a test sightline.

Here, in addition to what we see above, we get a warning that some manditory parameters must be set in order to process the model. We'll get to that in a couple of steps.

In [ ]:
mod2 = synthpop.SynthPop('huston2025_defaults.synthpop_conf',
                         extinction_map_kwargs={'name':'maps_from_dustmaps', 
                                                'dustmap_name': 'marshall'},
                         chosen_bands = ['Bessell_U', 'Bessell_B', 'Bessell_V', 'Bessell_R', 'Bessell_I'],
                         maglim = ['Bessell_I', 22, "remove"],
                         post_processing_kwargs=[{"name": "ProcessDarkCompactObjects","remove": False}],
                         name_for_output='mod2test'
                        )

#### Step 2: initialize populations

In [ ]:
mod2.init_populations()

#### Step 3: run catalogs we specify here

Because we did not provide sight lines in our configuration, we must set them here. 

mod2.process_all() cannot be run

Here, we capture the resulting DataFrame and distance distribution from the process_location function.

In [ ]:
cat2, distr2 = mod2.process_location(l_deg=3, b_deg=-1, solid_angle=1e-3, solid_angle_unit='deg^2')


#### Step 4: examine our output
Let's take a look at our output catalog:

In [ ]:
cat2

Here's the distribution of distances for the stars in our catalog. We see many stars in the near dist that are bright and close. We also pick up a number of the brighter stars in the dense central galactic bulge. Stars beyond this are generally too dim for our magnitude limit.

In [ ]:
plt.hist(cat2.Dist, bins=np.arange(0,25,0.5));
plt.xlabel('Distance (kpc)'); plt.ylabel('N stars')